# Prueba para el extract Data

# Para la API

In [6]:
import pandas as pd
import logging
import json
import requests
import great_expectations as gx
from tqdm import tqdm  # Para la barra de progreso

# Configuración del logging
logging.basicConfig(level=logging.INFO)

def load_API_data():
    url = "https://api.eia.gov/v2/petroleum/pri/gnd/data/?frequency=weekly&data[0]=value&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=1000&api_key=bqwjaJLDl8NGnarM5gvFz7iDmIGNyKK47vtgmX91"
    
    # Llamada a la API
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        
        if 'response' in data and 'data' in data['response']:
            records = data['response']['data']
            df = pd.DataFrame(records)
            logging.info("Data successfully extracted from API.")
            return df
        else:
            logging.error("JSON structure does not contain the expected data.")
            raise ValueError("Invalid JSON structure")
    else:
        logging.error(f"Failed API call with status code {response.status_code}")
        raise Exception("API call failed")



def validate_dataframe(df):
    # Definir columnas esperadas
    cols = ["period", "series", "area-name", "product", "product-name", "process", "process-name", "series-description", "value", "duoarea", "units"]
    
    # Configurar y validar expectativas directamente
    validator = gx.from_pandas(df)
    
    # Expectativa: columnas esperadas
    validator.expect_table_columns_to_match_set(column_set=cols, exact_match=False)
    
    # Definir tipos de datos esperados para cada columna
    column_types = {
        "period": "object",  # Ajusta el tipo según tus datos
        "area-name": "object",
        "product": "object",
        "product-name": "object",
        "process": "object",
        "process-name": "object",
        "series-description": "object",
        "value": "object",
        "duoarea": "object",
        "units": "object",
        "series": "object"
    }
    
    # Inicializar contadores para expectativas exitosas y totales
    total_expectations = 0
    successful_expectations = 0

    # Añadir expectativas para cada columna sobre tipo y no nulo, excepto para 'value'
    for column, dtype in column_types.items():
        # Excluir la columna 'value' de la validación de nulos
        if column == "value":
            total_expectations += 1  # Solo validamos el tipo de la columna 'value'
            type_check = validator.expect_column_values_to_be_of_type(column, dtype)
            if type_check.success:
                successful_expectations += 1
        else:
            total_expectations += 2  # Expectativas de tipo y no nulo para las demás columnas

            # Validación de tipo y nulos
            type_check = validator.expect_column_values_to_be_of_type(column, dtype)
            null_check = validator.expect_column_values_to_not_be_null(column)

            # Si cada expectativa pasa, aumenta el contador
            if type_check.success:
                successful_expectations += 1
            if null_check.success:
                successful_expectations += 1
    
    # Calcular porcentaje de expectativas exitosas
    success_percentage = (successful_expectations / total_expectations) * 100

    # Mostrar resultados
    if success_percentage == 100:
        logging.info("Todas las expectativas se validaron correctamente.")
    else:
        logging.warning(f"{success_percentage:.2f}% de expectativas exitosas.")

    # Convertir el resultado a un diccionario JSON serializable
    result_dict = validator.validate().to_json_dict()

    # Mostrar los resultados de validación en un formato legible
    print(json.dumps(result_dict, indent=4))
    print(f"Porcentaje de expectativas exitosas: {success_percentage:.2f}%")

# Llamar a la función de validación, proporcionando el DataFrame
# Ejemplo de cómo llamar a la función
# success_percentage = validate_data_with_expectations(df)
df = load_API_data()
validate_dataframe(df)


INFO:root:Data successfully extracted from API.
INFO:root:Todas las expectativas se validaron correctamente.
INFO:great_expectations.data_asset.data_asset:	22 expectation(s) included in expectation_suite.


{
    "success": true,
    "results": [
        {
            "success": true,
            "expectation_config": {
                "expectation_type": "expect_table_columns_to_match_set",
                "kwargs": {
                    "column_set": [
                        "period",
                        "series",
                        "area-name",
                        "product",
                        "product-name",
                        "process",
                        "process-name",
                        "series-description",
                        "value",
                        "duoarea",
                        "units"
                    ],
                    "exact_match": false,
                    "result_format": "BASIC"
                },
                "meta": {}
            },
            "result": {
                "observed_value": [
                    "period",
                    "duoarea",
                    "area-name",
                    "p

Al hacerse esta validación de datos en el proceso de extracción (Previo a la transformación), Tener en cuenta:

- Para estas expectativas, se tuvo en cuenta que todas las columnas fueran tipo object (Debido a que así son recuperadas originalmente/antes de la transformación)
- La columna Value presenta valores nulos (Como se explicó en el EDA) y por ende no se tiene en cuenta cuando se revisan que las columnas no tengan nulos